In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pprint import pprint

In [30]:
feature_df = pd.read_csv('features.csv')

X_train = pd.read_csv('train_lstm.csv')
X_test = pd.read_csv('test_lstm.csv')
y_train = pd.read_csv('train_labels.csv')
y_test = pd.read_csv('test_labels.csv')

print(f'X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}')

X_train: (3416, 41), X_test: (855, 41), y_train: (3416, 2), y_test: (855, 2)


In [31]:
import pandas_market_calendars as mcal

for df in (feature_df, y_train, y_test):
    df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
    df.set_index('date', inplace=True)

for df in (X_train, X_test):
    df['date'] = pd.to_datetime(df['date'], format = '%d/%m/%Y')
    df.set_index('date', inplace=True)

nyse = mcal.get_calendar('NYSE')
bday_us = pd.offsets.CustomBusinessDay(holidays=nyse.adhoc_holidays, calendar=nyse.regular_holidays, weekmask="1111100")
feature_df = feature_df.asfreq(bday_us)
X_train = X_train.asfreq(bday_us)
X_test = X_test.asfreq(bday_us)
y_train = y_train.asfreq(bday_us)['ERP_Sign']
y_test = y_test.asfreq(bday_us)['ERP_Sign']

print(f'X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}')

X_train: (3416, 40), X_test: (855, 40), y_train: (3416,), y_test: (855,)


In [53]:
data_df = pd.concat([X_train, X_test])

In [20]:
max_lags = 10
fcst_horizon = [5, 10, 15, 20]
model_vars = feature_df.columns

In [73]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.metrics import accuracy_score, average_precision_score, f1_score, precision_score, recall_score, roc_auc_score
from sktime.utils.plotting import plot_series

tf.random.set_seed(100)

lstm_train_df = pd.DataFrame(
    [(f'h{h}', 'lstm_clf') for h in fcst_horizon], columns=['fh','clf']    
)

lstm_test_df = pd.DataFrame(
    [(f'h{h}', 'lstm_clf') for h in fcst_horizon], columns=['fh','clf']    
)

lstm_pred_df = pd.DataFrame().reindex(data_df.index)
lstm_prob_df = pd.DataFrame().reindex(data_df.index)

k = 0
for h in fcst_horizon:
    p = max_lags + h
    X_train_h = []
    y_train_h = []
    for i in range(p, len(X_train)):
        X_train_h.append(X_train[i-p:i-h])
        y_train_h.append(y_train[i])
    X_train_h, y_train_h = np.array(X_train_h), np.array(y_train_h)

    X_train_h = np.reshape(X_train_h, (X_train_h.shape[0], X_train_h.shape[1], 40))

    lstm_clf = Sequential()

    lstm_clf.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_h.shape[1], 40)))
    lstm_clf.add(Dropout(0.2))

    lstm_clf.add(LSTM(units=50, return_sequences=True))
    lstm_clf.add(Dropout(0.2))

    lstm_clf.add(LSTM(units=50, return_sequences=True))
    lstm_clf.add(Dropout(0.2))

    lstm_clf.add(LSTM(units=50))
    lstm_clf.add(Dropout(0.2))

    lstm_clf.add(Dense(1, activation='sigmoid'))

    lstm_clf.compile(
        optimizer='adam', loss='binary_crossentropy', metrics=['acc']
    )

    lstm_clf.fit(X_train_h, y_train_h, epochs=200)

    y_pred_train = lstm_clf.predict(X_train_h)
    y_pred_train[y_pred_train <= 0.5] = 0
    y_pred_train[y_pred_train > 0.5] = 1
    y_prob_train = lstm_clf.predict(X_train_h)

    lstm_train_df.loc[i, 'Accuracy'] = accuracy_score(y_train_h, y_pred_train)
    lstm_train_df.loc[i, 'F1 Score'] = f1_score(y_train_h, y_pred_train)
    lstm_train_df.loc[i, 'PR AUC'] = average_precision_score(y_train_h, y_prob_train)
    lstm_train_df.loc[i, 'Precision'] = precision_score(y_train_h, y_pred_train)
    lstm_train_df.loc[i, 'Sensitivity'] = recall_score(y_train_h, y_pred_train)
    lstm_train_df.loc[i, 'Specificity'] = recall_score(y_train_h, y_pred_train, pos_label=0)
    lstm_train_df.loc[i, 'ROC AUC'] = roc_auc_score(y_train_h, y_prob_train)

    X_test_h = []
    for i in range(len(X_train), len(data_df)):
        X_test_h.append(data_df[i-p:i-h])
    X_test_h = np.array(X_test_h)
    X_test_h = np.reshape(X_test_h, (X_test_h.shape[0], X_test_h.shape[1], 40))

    y_pred_test = lstm_clf.predict(X_test_h)
    y_pred_test[y_pred_test <= 0.5] = 0
    y_pred_test[y_pred_test > 0.5] = 1
    y_prob_test = lstm_clf.predict(X_test_h)
    
    lstm_test_df.loc[i, 'Accuracy'] = accuracy_score(y_test, y_pred_test)
    lstm_test_df.loc[i, 'F1 Score'] = f1_score(y_test, y_pred_test)
    lstm_test_df.loc[i, 'PR AUC'] = average_precision_score(y_test, y_prob_test)
    lstm_test_df.loc[i, 'Precision'] = precision_score(y_test, y_pred_test)
    lstm_test_df.loc[i, 'Sensitivity'] = recall_score(y_test, y_pred_test)
    lstm_test_df.loc[i, 'Specificity'] = recall_score(y_test, y_pred_test, pos_label=0)
    lstm_test_df.loc[i, 'ROC AUC'] = roc_auc_score(y_test, y_prob_test)

    # Convert np arrays to datetime series for time series plotting
    time_start = y_train.index[0] + p*bday_us
    mask_ystart = y_train.index >= time_start
    ytrain_h = y_train.loc[mask_ystart]
    y_pred = pd.concat([pd.Series(y_pred_train.flatten()).set_axis(ytrain_h.index), pd.Series(y_pred_test.flatten()).set_axis(y_test.index)])
    y_prob = pd.concat([pd.Series(y_prob_train.flatten()).set_axis(ytrain_h.index), pd.Series(y_prob_test.flatten()).set_axis(y_test.index)])

    lstm_pred_df[(f'h{h}', 'lstm_clf')] = y_pred
    lstm_prob_df[(f'h{h}', 'lstm_clf')] = y_prob

    # Plot the actual recession binary values with the predicted binary values
    plot_series(ytrain_h, y_test, y_pred, labels=['y_train', 'y_test', 'y_pred'], x_label='Date', y_label='ERP Sign (Binary)')
    plt.title(f'erp_binary_lstm_clf_h{h}')
    plt.tight_layout()
    plt.savefig(f'erp_binary_lstm_clf_h{h}.png')
    plt.close()

    # Plot the actual recession binary values with the predicted probabilities
    plot_series(ytrain_h, y_test, y_prob, labels=['y_train', 'y_test', 'y_prob'], x_label='Date', y_label='Pr(ERP Sign)')
    plt.title(f'erp_prob_lstm_clf_h{h}')
    plt.tight_layout()
    plt.savefig(f'erp_prob_lstm_clf_h{h}.png')
    plt.close()

    k += 1

Epoch 1/200
107/107 [==============================] - 12s 35ms/step - loss: 0.6940 - acc: 0.5166
Epoch 2/200
107/107 [==============================] - 3s 33ms/step - loss: 0.6919 - acc: 0.5178
Epoch 3/200
107/107 [==============================] - 4s 33ms/step - loss: 0.6934 - acc: 0.5107
Epoch 4/200
107/107 [==============================] - 4s 37ms/step - loss: 0.6923 - acc: 0.5169
Epoch 5/200
107/107 [==============================] - 4s 39ms/step - loss: 0.6916 - acc: 0.5231
Epoch 6/200
107/107 [==============================] - 5s 43ms/step - loss: 0.6913 - acc: 0.5210
Epoch 7/200
107/107 [==============================] - 4s 37ms/step - loss: 0.6918 - acc: 0.5196
Epoch 8/200
107/107 [==============================] - 4s 38ms/step - loss: 0.6914 - acc: 0.5269
Epoch 9/200
107/107 [==============================] - 5s 43ms/step - loss: 0.6920 - acc: 0.5248
Epoch 10/200
107/107 [==============================] - 4s 35ms/step - loss: 0.6908 - acc: 0.5146
Epoch 11/200
107/107 [======

In [105]:
lstm_train_df.to_csv('erp_lstm_train_results.csv')
lstm_test_df.to_csv('erp_lstm_test_results.csv')
lstm_pred_df.to_csv('erp_lstm_pred_estimates.csv')
lstm_prob_df.to_csv('erp_lstm_prob_estimates.csv')